### 네이버 영화 리뷰 감성분석 Ver.2
- tokenizer 함수
- TfidfVectorizer + NaiveBayes

#### 중요 dropna시 전체 프레임을 잡아야한다.!!!
- 데이터 다운로드 링크 : https://github.com/e9t/nsmc/

In [6]:
import pandas as pd
train_df = pd.read_csv('data/네이버영화리뷰 train 전처리완료.tsv', sep='\t')
test_df = pd.read_csv('data/네이버영화리뷰 test 전처리완료.tsv', sep='\t')

In [7]:
X_train, y_train = train_df.document.values, train_df.label.values
X_test, y_test = test_df.document.values, test_df.label.values

- Tokenizer 함수

In [8]:
with open('data/한글불용어100.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]

In [9]:
from konlpy.tag import Okt
okt = Okt()

In [10]:
def okt_tokenizer(text):
    morphs = okt.morphs(text, stem=True)
    tokens = [morph for morph in morphs if morph not in stop_words]
    return tokens

- Pipeline 학습

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [13]:
# token_pattern = default str = r"(?u)\b\w\w+\b" 분리되어 있는 놈을 쓰니까! None
pipeline = Pipeline([
    ('tvect', TfidfVectorizer(tokenizer=okt_tokenizer, token_pattern=None)), # 내것을 쓰니까
    ('nb', MultinomialNB())
])
%time pipeline.fit(X_train, y_train)

CPU times: total: 12min 18s
Wall time: 12min 1s


Pipeline(steps=[('tvect',
                 TfidfVectorizer(tokenizer=<function okt_tokenizer at 0x00000235E3BE6160>)),
                ('nb', MultinomialNB())])

In [14]:
%time pipeline.score(X_test, y_test)

CPU times: total: 3min 48s
Wall time: 3min 44s


0.8356464423155654

- 실제 데이터 적용

In [15]:
reviews = [
    '이 영화 개꿀잼 ㅋㅋㅋ', '이 영화 핵노잼 ㅠㅠ', '이딴게 영화냐 ㅉㅉ',
    '감독 뭐하는 놈이냐?', '와 개쩐다 정말 세계관 최강자들의 영화다'
]

In [16]:
# 전처리 - 한글 이외의 문자는 공백으로 변환
import re
reviews = list(map(lambda x : re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', x), reviews))
reviews

['이 영화 개꿀잼 ㅋㅋㅋ',
 '이 영화 핵노잼 ㅠㅠ',
 '이딴게 영화냐 ㅉㅉ',
 '감독 뭐하는 놈이냐 ',
 '와 개쩐다 정말 세계관 최강자들의 영화다']

In [17]:
pipeline.predict(reviews)

array([1, 0, 0, 0, 1], dtype=int64)

- 최적 파라메터 찾기
    - tokenizer를 설정하지 말고, 형태소 분석이 끝난 데이터로 수행할 것.
    - [ , ] [ , ] cv=3 2\*2\*3 = 12회 필요한데 토크나이저를 뺀다.


In [11]:
import pandas as pd
train_df = pd.read_csv('data/네이버영화리뷰 train 형태소처리완료.tsv')
test_df = pd.read_csv('data/네이버영화리뷰 test 형태소처리완료.tsv')

In [12]:
train_df.isna().sum()

X    51
y     0
dtype: int64

In [13]:
test_df.isna().sum()

X    24
y     0
dtype: int64

In [14]:
train_df.dropna(how='any', inplace=True)
test_df.dropna(how='any', inplace=True)

In [15]:
train_df.isna().sum()

X    0
y    0
dtype: int64

In [16]:
X_train, y_train = train_df.X.values, train_df.y.values
X_test, y_test = test_df.X.values, test_df.y.values

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
pipeline = Pipeline([
    ('TVECT', TfidfVectorizer()), ('NB', MultinomialNB())
])
params = {'TVECT__ngram_range' : [(1,1), (1,2)], 'TVECT__max_df' : [0.95, 0.98]}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)

In [18]:
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 40.2 s
Wall time: 40.5 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('TVECT', TfidfVectorizer()),
                                       ('NB', MultinomialNB())]),
             param_grid={'TVECT__max_df': [0.95, 0.98],
                         'TVECT__ngram_range': [(1, 1), (1, 2)]},
             scoring='accuracy')

In [19]:
grid_pipe.best_params_

{'TVECT__max_df': 0.95, 'TVECT__ngram_range': (1, 2)}

In [20]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.846194806258704

- 모델 저장하기

In [19]:
import joblib
joblib.dump(pipeline, 'model/네이버영화리뷰_pipeline.pkl')

['model/네이버영화리뷰_pipeline.pkl']